# CIFAR 10

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## ResNet v2
### EDA

In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
cifar10 = keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

unique, counts = np.unique(y_train, return_counts=True)
print(f"Train Labels: {dict(zip(unique, counts))}")
unique, counts = np.unique(y_test, return_counts=True)
print(f"Test Labels: {dict(zip(unique, counts))}")

Train Labels: {0: 5000, 1: 5000, 2: 5000, 3: 5000, 4: 5000, 5: 5000, 6: 5000, 7: 5000, 8: 5000, 9: 5000}
Test Labels: {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}


### Modeling

In [4]:
from keras.layers import Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Flatten, Dense
from keras.regularizers import l2

In [5]:
batch_size = 32 # origin paper trained all networks with bath_size=128
epochs = 200
data_augmentation = True
num_classes = len(np.unique(y_train))

n=12

# Model version
version = 2

# Compute depth from supplied model parameter n
if version == 1:
    depth = n*6+2
elif version == 2:
    depth = n*9+2

# Model name, depth and version
model_type = 'ResNet%d_v%d'%(depth, version)

# Input image dimensions
input_shape = x_train.shape[1:]

# One-hot Encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Normalize data
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# Subtracting pixel mean improves accuracy
subtract_pixel_mean=True
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)


In [6]:
def lr_schedule(epoch):
    # Learning Rate Schedule
    
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print(f"Learning rate: {lr}")
    return lr

In [7]:
def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1, activation='relu', batch_normalization=True, conv_first=True):
    # Conv2D-BN-Activation stack builder
    # inputs (tensor) : imput image or previous layer
    # num_filters (int) : Conv2D number of filters
    # kernel_size (int) : Conv2D square kernel dimensions
    # strides (int) : Conv2D square stride dimention
    # activation (string) : Activation name
    # batch_normalization (bool) : wWhether to include BN
    # conv_first (bool) : Conv2D-BN-Activation (True) or bn-activation-conv (False)
    # Returns : x (tensor) as input to the next layer
    
    conv = Conv2D(filters=num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4)
                 )
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x =Activation(activation)(x)
        x = conv(x)
    return x

In [8]:
def resnet_v2(input_shape, depth, num_classes=10):
    if (depth-2)%9 != 0:
        raise ValueError('depth should be 9n+2 (eg 20, 32, 44 in [a])')
        
    num_filters_in = 16
    num_res_blocks = int((depth-2)/9)
    
    inputs = keras.Input(shape = input_shape)
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True
                    )
    
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:
                    strides = 2
                    
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False
                            )
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False
                            )
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False
                            )
            if res_block == 0:
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False
                                )
            x = keras.layers.add([x, y])
        num_filters_in *= num_filters_out
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(y)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
    
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

Metal device set to: Apple M2


2023-02-04 09:41:45.440929: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-04 09:41:45.441254: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Mul: Dst tensor is not initialized. [Op:Mul]

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy']
             )
model.summary()